In [2]:
import pandas as pd

df = pd.read_csv('../data/dataset/tutas_recommender_training_features.csv')

print(df.head())
df.info()
df.isnull().sum()


  id_murid id_tutor  topik_match  gaya_match  metode_match  time_overlap  \
0    M0566    T0008         True        True          True         False   
1    M0379    T0013         True        True          True         False   
2    M0964    T0013         True        True          True         False   
3    M0100    T0017         True        True          True         False   
4    M0416    T0018         True        True          True         False   

   murid_flexible  tutor_flexible  feedback_score  label  \
0           False           False               0      0   
1           False           False               0      0   
2           False           False               0      0   
3           False           False               0      0   
4           False           False               0      0   

   average_rating_tutor  total_jam_ngajar_tutor  pernah_gagal  
0                  3.71                      72          True  
1                  4.60                     323       

id_murid                  0
id_tutor                  0
topik_match               0
gaya_match                0
metode_match              0
time_overlap              0
murid_flexible            0
tutor_flexible            0
feedback_score            0
label                     0
average_rating_tutor      0
total_jam_ngajar_tutor    0
pernah_gagal              0
dtype: int64

In [3]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

print("Fitur Numerikal : ", list(numerical_cols))
print(df[numerical_cols].describe())

Fitur Numerikal :  ['feedback_score', 'label', 'average_rating_tutor', 'total_jam_ngajar_tutor']
       feedback_score        label  average_rating_tutor  \
count     2012.000000  2012.000000           2012.000000   
mean         3.519384     0.736581              3.973335   
std          1.920116     0.440597              0.574820   
min          0.000000     0.000000              3.000000   
25%          3.000000     0.000000              3.490000   
50%          4.000000     1.000000              3.960000   
75%          5.000000     1.000000              4.452500   
max          5.000000     1.000000              5.000000   

       total_jam_ngajar_tutor  
count             2012.000000  
mean               196.456759  
std                109.466772  
min                 11.000000  
25%                106.000000  
50%                193.000000  
75%                287.000000  
max                400.000000  


In [4]:
# Sekarang kita akan scale pakai standard scaler karena data distribusinya normal dan tidak banyak outlier, kalau banyak outlier, kita harus pakai RobustScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('../data/dataset/tutas_recommender_training_features.csv')

X = df.drop(columns=['label'])
y = df['label']

numerical_cols = ['feedback_score', 'average_rating_tutor', 'total_jam_ngajar_tutor']

scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])




In [7]:
# Sekarang kita split untuk data training dan testing. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.to_csv('../data/dataset/processed/X_train.csv', index=False)
X_test.to_csv('../data/dataset/processed/X_test.csv', index=False)
y_train.to_csv('../data/dataset/processed/y_train.csv', index=False)
y_test.to_csv('../data/dataset/processed/y_test.csv', index=False)



